*This Jupyter notebook merges donors datasets and creates a table at the municipality-year of election level with the average number of donors and average donation amount*.

In [1]:
#Import packages 
import os
import pandas as pd

In [2]:
#Change working directory 
os.chdir('D:/Users/USER/Documents/UR 2021-2/MCPP/Project/donations_in_peace')

In [3]:
#Load donors datasets and check 5 rows of 2015 to see how the data is structured.
donors2011 = pd.read_csv('data/auxiliar/names/donors_2011.csv')
donors2015 = pd.read_csv('data/auxiliar/names/donors_2015.csv')
donors2019 = pd.read_csv('data/auxiliar/names/donors_2019.csv')
donors2011.head()

,name,amount,candidate,corp,dpto,mpio,party,codmpio,coddpto
0,HORACIO GUARDIA MACHADO,10748000.0,ANIBAL GAVIRIA CORREA,Alcaldía,ANTIOQUIA,MEDELLIN,PARTIDO LIBERAL COLOMBIANO,5001.0,5.0
1,JOHNNY BAIZ MATUK,15000000.0,ANIBAL GAVIRIA CORREA,Alcaldía,ANTIOQUIA,MEDELLIN,PARTIDO LIBERAL COLOMBIANO,5001.0,5.0
2,JOSE E MUÑOZ,3000000.0,ANIBAL GAVIRIA CORREA,Alcaldía,ANTIOQUIA,MEDELLIN,PARTIDO LIBERAL COLOMBIANO,5001.0,5.0
3,RAFAEL LINARES OIRJUELA,23265000.0,ANIBAL GAVIRIA CORREA,Alcaldía,ANTIOQUIA,MEDELLIN,PARTIDO LIBERAL COLOMBIANO,5001.0,5.0
4,BEATRIZ RESTREPO,1000000.0,ANIBAL GAVIRIA CORREA,Alcaldía,ANTIOQUIA,MEDELLIN,PARTIDO LIBERAL COLOMBIANO,5001.0,5.0


In [4]:
#Load candidates names datasets
candidates2015 = pd.read_csv('data/auxiliar/allcandidatesnames15.csv')
candidates2019 = pd.read_csv('data/auxiliar/allcandidatesnames19.csv')
print(len(candidates2015))
print(len(candidates2019))

4562
103347


In [5]:
#There are some repetitions of unique identifiers in the candidates datasets. Keep the first one.
candidates2019 = candidates2019.drop_duplicates(subset=['wp_no']) #16 observations dropped. 
candidates2015 = candidates2015.drop_duplicates(subset=['wp_no']) #347 observations dropped.
print(len(candidates2015))
print(len(candidates2019))

4562
103331


In [6]:
#Keep only candidates for mayor (alcaldias)
candidates2015=candidates2015[candidates2015.corp=="Alcaldía"]
candidates2019=candidates2019[candidates2019.corp=="Alcaldía"]
print(len(candidates2015))
print(len(candidates2019))

4562
5142


**Resultado**:
- En 2015 hubo 4,193 candidatos a Alcaldía
- En 2019: 5,142

In [7]:
#Rename variables 'name' and 'wp_no' in the candidates datasets to 'candidate_name' and 'cand_number'
candidates2015=candidates2015.rename(columns={"name": "candidate_name", "wp_no": "cand_number"})
candidates2019=candidates2019.rename(columns={"name": "candidate_name", "wp_no": "cand_number"})

In [8]:
#Match donors with candidates names by candidate number. Then, get the number of rows in each dataset.
#This merge only applies for 2015 and 2019. For 2011, the donors are already matched. 
donors2015_matched = donors2015.merge(candidates2015, on='cand_number')
donors2019_matched = donors2019.merge(candidates2019, on='cand_number')
print(len(donors2015_matched))
print(len(donors2019_matched))

8007
12553


In [9]:
#Rename 'candidate' as 'candidate_name' in 2011
donors2011=donors2011.rename(columns={"candidate": "candidate_name"})

In [10]:
#Compute the total amount of donation by municipality-candidate-donor
donors2015_matched_by_donors=donors2015_matched.groupby(['dpto','mpio','candidate_name','name']).sum()
donors2019_matched_by_donors=donors2019_matched.groupby(['dpto','mpio','candidate_name','name']).sum()
donors2011_matched_by_donors=donors2011.groupby(['codmpio','candidate_name','name']).sum()
donors2011_matched_by_donors.head(5)

amount  coddpto
codmpio candidate_name        name                                     
5001.0  ANIBAL GAVIRIA CORREA 3PONINTECH S.A.S        562600.0      5.0
                              ALFREDO LOPEZ          9900000.0      5.0
                              ALMACENES ÉXITO S.A.  32000000.0      5.0
                              ALZATE NOREÑA          3320206.0      5.0
                              BEATRIZ RESTREPO       1000000.0      5.0

In [11]:
#Compute the number of donors and total amount of donation by municipality-candidate
donors2011_matched_by_cand=donors2011_matched_by_donors.groupby(['codmpio','candidate_name']).agg({'amount':'sum','coddpto':'count'})
donors2015_matched_by_cand=donors2015_matched_by_donors.groupby(['dpto','mpio','candidate_name']).agg({'amount':'sum','cand_number':'count'})
donors2019_matched_by_cand=donors2019_matched_by_donors.groupby(['dpto','mpio','candidate_name']).agg({'amount':'sum','cand_number':'count'})

In [12]:
#Rename coddpto and cand_number as 'no_of_donors'
donors2011_matched_by_cand=donors2011_matched_by_cand.rename(columns={"coddpto": "no_of_donors"})
donors2015_matched_by_cand=donors2015_matched_by_cand.rename(columns={"cand_number": "no_of_donors"})
donors2019_matched_by_cand=donors2019_matched_by_cand.rename(columns={"cand_number": "no_of_donors"})

In [13]:
#Compute the average and total number of donors, as well as amount, by municipality.
mpio_stats_2011=donors2011_matched_by_cand.groupby(['codmpio']).agg({'amount':['sum', 'mean'],'no_of_donors':['sum', 'mean']})
mpio_stats_2015=donors2015_matched_by_cand.groupby(['dpto','mpio']).agg({'amount':['sum', 'mean'],'no_of_donors':['sum', 'mean']})
mpio_stats_2019=donors2019_matched_by_cand.groupby(['dpto','mpio']).agg({'amount':['sum', 'mean'],'no_of_donors':['sum', 'mean']})
#See the final dataset
mpio_stats_2015.head()

amount               no_of_donors     
                                sum          mean          sum mean
dpto      mpio                                                     
AMAZONAS  LETICIA        13000000.0  1.300000e+07            2  2.0
          PUERTO NARIÑO   3050000.0  1.525000e+06            2  1.0
ANTIOQUIA ABEJORRAL      59500000.0  1.983333e+07            9  3.0
          AMAGA          15000000.0  1.500000e+07            3  3.0
          AMALFI         65904879.0  3.295244e+07           17  8.5

In [14]:
#Export final dataset in CSV format
mpio_stats_2011.to_csv('data/clean/mpio_stats_2011.csv')
mpio_stats_2015.to_csv('data/clean/mpio_stats_2015.csv')
mpio_stats_2019.to_csv('data/clean/mpio_stats_2019.csv')